In [8]:
import json
import csv
import tweepy
import re
from datetime import date,datetime,timedelta

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import yfinance as yf

## 1. Twitter API:

In [41]:
def search_for_hashtags(hashtag_phrase, date_since, end_date):
    #auth = tweepy.OAuthHandler("psxikKIcDvu19SDl1qbTycOHY", "H2FV5HL4UyuzVUtANd0lo3HpsNzo1woDMuwlabVyv2E7g44SDb")
    #auth.set_access_token("1265371572942102528-XZyshdLemub7C7hrx51dyFrYCBCmvU", "CK4hmNukUYBhzr3CAtO3gvGFx7Ahr9TW3vZMjW6pl8yVd")

    auth = tweepy.OAuthHandler("4BSOA2XKS8vucCHyBjy502Aw8", "l27Zsdh9X9oGGff25gJ4PX9zN6ZnjYKonu2zISu17jsQlO5Dkb")
    auth.set_access_token("1265371572942102528-IsosHWjrXRDKHaqQFAbSPuM2FyH41k", "pVjYFAx8pDCBnJE48NxsA4KB6g4eNMw39TOWcTciLOB5u")

    #auth = tweepy.OAuthHandler("dQXGlGv8YFtPrZvQwcCnvbged", "ouKbQUg1dGubyKbp7DrNo45Qdv3nNzd7MyuvCgIKha0vpuNbDA")
    #auth.set_access_token("1265371572942102528-Jmu9hvd4yBep0KwV9U5mHFUnyUi9JV", "CYW3FRkyXJnSzRrHoN9FcLlBdSRHfA7GWJK1PMT7Q1S7P")

    #auth = tweepy.OAuthHandler("ljU1UWBCC0YNKlO9pwm1TshUc", "w5CDP6fNeNQhvDs075KiZQEIWI7VY1Z8BxiDc5kUAHTsXzOhCY")
    #auth.set_access_token("1309241898419322880-hBG39tjNql0FcHrjOMYg2qTN3TEtnw", "p0UOsVpTB9hu15R3YBMbWGw2zDpJjkUCmK0YadzA894ZF")

    api = tweepy.API(auth)

    with open('%s.csv' % (hashtag_phrase[1:] + '_' + start_date), 'w+') as file:
        w = csv.writer(file)
        w.writerow(['timestamp', 'tweet_text', 'followers_count'])
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', \
                                   lang="en", tweet_mode='extended', since=start_date, until=end_date).items():
            w.writerow([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8'), tweet.user.followers_count])

In [42]:
#consumer_key = input('Consumer Key: ')
#consumer_secret = input('Consumer Secret: ')
#access_token = input('Access Token: ')
#access_token_secret = input('Access Token Secret: ')
    
hashtag_phrase = input('Hashtag Phrase: ')
start_date = input('Start Date: ')
end_date = input('End Date: ')

if __name__ == '__main__':
    search_for_hashtags(hashtag_phrase, start_date, end_date)

## 2. News Headlines:

In [14]:
def get_news(ticker_code):
    # 1. Define URL:
    finwiz_url = 'https://finviz.com/quote.ashx?t='
    # 2. Requesting data:
    news_tables = {}
    tickers = [ticker_code]
    for ticker in tickers:
        url = finwiz_url + ticker
        req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
        response = urlopen(req)    
        # Read the contents of the file into 'html'
        html = BeautifulSoup(response)
        # Find 'news-table' in the Soup and load it into 'news_table'
        news_table = html.find(id='news-table')
        # Add the table to our dictionary
        news_tables[ticker] = news_table
    #3. Parsing news:
    parsed_news = []
    # Iterate through the news
    for file_name, news_table in news_tables.items():
        # Iterate through all tr tags in 'news_table'
        for x in news_table.findAll('tr'):
            # read the text from each tr tag into text
            # get text from a only
            text = x.a.get_text() 
            # splite text in the td tag into a list 
            date_scrape = x.td.text.split()
            # if the length of 'date_scrape' is 1, load 'time' as the only element

            if len(date_scrape) == 1:
                time = date_scrape[0]
                
            # else load 'date' as the 1st element and 'time' as the second    
            else:
                date = date_scrape[0]
                time = date_scrape[1]
            # Extract the ticker from the file name, get the string up to the 1st '_'  
            ticker = file_name.split('_')[0]
            
            # Append ticker, date, time and headline as a list to the 'parsed_news' list
            parsed_news.append([ticker, date, time, text])

    # 4. Split into columns and save:
    # Instantiate the sentiment intensity analyzer
    vader = SentimentIntensityAnalyzer()
    # Set column names
    columns = ['ticker', 'date', 'time', 'headline']
    # Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
    parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)
    # Iterate through the headlines and get the polarity scores using vader
    scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()
    # Convert the 'scores' list of dicts into a DataFrame
    scores_df = pd.DataFrame(scores)
    # Join the DataFrames of the news and the list of dicts
    parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')
    # Convert the date column from string to datetime
    parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date
    parsed_and_scored_news.to_csv('~/LighthouseLabs-Final/2. FinViz_Headline_Data/' + ticker + '_data_' + (datetime.today().strftime('%Y-%m-%d-%H-%M')) + '.csv')

In [3]:
get_news('AAPL'), get_news('TSLA'), get_news('AMZN'), get_news('GS'), get_news('FB'), get_news('MSFT'), get_news('NVDA'), get_news('JNJ'), get_news('NFLX'), get_news('CVX'), get_news('PFE'), get_news('NKE'), get_news('GOOG')

print("Done!")

Done!


## 3. Historical Stock Data:

In [11]:
def stock_data(ticker):
    start_date = '2020-09-23'
    end_date = date.today()+timedelta(days=1)
    # 1. Request data:
    data = yf.download(ticker, 
                      start=start_date, 
                      end=end_date,
                      interval='30m', 
                      progress=False)
    # 2. Feature Engineering:
    data['Percent Price Change Within Period'] = ((data['Close'] - data['Open'])/data['Open'])*100
    data['Change in Close Price'] = data['Close'] - data['Close'].shift(1)
    data['Scaled Volume'] = data['Volume']/data['Volume'].mean()
    data_SMA = data['Adj Close'].rolling(window=3).mean().shift(1)
    data['SMA(3)'] = data_SMA
    data.reset_index(inplace=True)
    data['Datetime']=data['Datetime'].dt.tz_convert('America/Montreal').dt.tz_localize(None)
    #3. Export data:
    f_name = ticker + "_data"
    data.to_csv('~/LighthouseLabs-Final/1. Stock_Data/' + f_name + ".csv")
    print('Data saved!')

In [12]:
stock_data('AAPL'), stock_data('GOOG'), stock_data('FB'), stock_data('AMZN'), stock_data('NFLX'), stock_data('TSLA')

stock_data('JNJ'), stock_data('MSFT'), stock_data('CVX'), stock_data('NKE'), stock_data('PFE'), stock_data('GS'), stock_data('NVDA')


Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!


(None, None, None, None, None, None, None)